In [1]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import Dataset

from utils import *
from dataset import *
from preprocess import *
from wrapper import *
from models import BertWithNER, AutoModelWithNER

# device = torch.device("cpu")
device = torch.device("cuda:0")

In [4]:
train_df = pd.read_csv('../data/train.csv', sep='\t', index_col='id')
test_df = pd.read_csv('../data/test.csv', sep='\t', index_col='id')

model_name = 'hfl/chinese-macbert-base'
ner_model_name = 'uer/roberta-base-finetuned-cluener2020-chinese'

test_dataset_config = {
    'model_name':model_name,
    'aux_model_name':ner_model_name,
    'maxlength':128,
    'train_val_split':-1,
    'test':True, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'device':device,
}

train = DatasetWithAuxiliaryEmbeddings(df=test_df.reset_index(), **test_dataset_config)
train.tokenize()
train.construct_dataset()

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


In [5]:
checkpoints = [
    '../ner_run/fold0/checkpoint-5092/pytorch_model.bin', 
    # '../ner_run_best/fold1/checkpoint-7425/pytorch_model.bin', 
    # '../ner_run_best/fold2/checkpoint-9900/pytorch_model.bin', 
    # '../ner_run_best/fold3/checkpoint-7425/pytorch_model.bin', 
    # '../ner_run_best/fold4/checkpoint-4950/pytorch_model.bin', 
    # '../ner_run_best/fold5/checkpoint-9900/pytorch_model.bin', 
    # '../ner_run_best/fold6/checkpoint-9900/pytorch_model.bin', 
    # '../ner_run_best/fold7/checkpoint-9900/pytorch_model.bin', 
]

# model = BertWithNER(bert_model=model_name, ner_model=ner_model_name)
model = AutoModelWithNER(model=model_name, ner_model=ner_model_name)
state_dict = torch.load(checkpoints[0], map_location=device)
# for key in list(state_dict.keys()):
#     state_dict[key.replace('bert', 'base_model')] = state_dict.pop(key)
model.load_state_dict(state_dict)

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at uer/roberta-base-finetuned-cluener2020-chinese were not used when initializi

<All keys matched successfully>

In [6]:
output_tensors = []

for cp in checkpoints:
    # model = BertWithNER(bert_model=model_name, ner_model=ner_model_name)
    model = AutoModelWithNER(model=model_name, ner_model=ner_model_name)
    state_dict = torch.load(cp, map_location=device)
    # for key in list(state_dict.keys()):
    #     state_dict[key.replace('bert', 'base_model')] = state_dict.pop(key)
    model.load_state_dict(state_dict)

    logits = []
    dataloader = DataLoader(train.dataset['train'].with_format('torch'), batch_size=16)
    for batch in dataloader:
        outputs = model(**batch)
        logits.append(outputs['logits'])

    del model
    output_tensors.append(torch.concat(logits))

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at uer/roberta-base-finetuned-cluener2020-chinese were not used when initializi

In [102]:
train_df = pd.read_csv('../data/train.csv', sep='\t', index_col='id')

data = train_df[500:600]
data['prediction'] = torch.argmax(output_tensors[0], 1)
data = data[['label', 'prediction', 'text']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(data[data.label != data.prediction])

C:\Users\holaj\AppData\Local\Temp\ipykernel_29028\1890036286.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['prediction'] = torch.argmax(output_tensors[0], 1)


,label,prediction,text
id,,,
505,0,1,矗立在鲁迅纪念馆前的这座铜像，是由巴金等著名作家倡议，国内外热爱鲁迅的人 士集资30万元铸成的。
506,1,0,关于鸟类的进化，有一种观点认为，鸟最早用４个翅膀滑翔的，后来才进化成骨骼轻巧、拍动双翼的飞行...
513,0,1,消费者通过网络平台购买商品，其合法权益受到损害时，可以向销售者要求赔偿。
526,0,1,要实现可持续发展，就要依靠科学技术，形成少投入、多产出的生产方式和少排放、多利用的消费模式，...
529,0,1,杨利伟：第一位进入太空的中国航天员。
534,0,1,去年，对于石油投资者和生产者，是破天荒的一年，石油价格大幅下跌，不仅改写了石油业的历史，也打...
537,1,0,这家企业将携手社会力量，共同开展“健康中国”系列活动：构建健康生态、倡导健康生活、宣传健康理念。
539,0,1,作为消费者，你是否想过，正是某些畸形的消费需求刺激了商业经营对生态的污染和破坏，从这个意义上...
547,1,0,孔子一生积极求索，“知其不可为而为之”，奋斗一生的他是他这句格言的真实写照。


In [22]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in train.tokenizer.model_input_names or k == 'auxiliary_input_ids'}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output['logits'], axis=-1)
        loss = cross_entropy(output['logits'], batch["label"].to(device),
                             reduction="none")
    # Place outputs on CPU for compatibility with other dataset columns
    return {"loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()}

In [23]:
train.dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'auxiliary_input_ids'])
eval_outputs = train.dataset["val"].map(forward_pass_with_label, batched=True, batch_size=16)

  0%|          | 0/7 [00:00<?, ?ba/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

In [9]:
train.dataset.set_format('torch')
train.dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'auxiliary_input_ids', 'labels'],
        num_rows: 100
    })
})